In [908]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio

import coffee.color as c
import coffee.palletts as p


In [909]:
xls = pd.ExcelFile(
    # "./data/Student and Faculty Commons Survey_February 16, 2022_09.17.xlsx"
    "./data/Fakedata.xlsx"
)

# data = pd.read_excel( xls, "Student and Faculty Commons Sur" )
data = pd.read_excel( xls, "data" )

# useable_data = data.drop( [0, 1] )
# useable_data = useable_data.drop( data.columns[0:17], axis=1 )

intersections = data[ ["intersection", "columnName"] ]
useable_data = data

useable_data.fillna('No Response', inplace=True)    
intersections.fillna("blank", inplace=True)    




useable_data.head

/Users/brianmasse/opt/miniconda3/envs/CSC630/lib/python3.9/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<bound method NDFrame.head of     Are You a Student or Faculty Member?  What is Your Residency Status?   \
0                                 Faculty                             Day   
1                                 Faculty                     Boarding-US   
2                                 Student                     Boarding-US   
3                                 Faculty          Boarding-International   
4                                 Faculty                             Day   
..                                    ...                             ...   
995                               Student                     Boarding-US   
996                               Faculty                     Boarding-US   
997                               Student                             Day   
998                               Student          Boarding-International   
999                           No Response                     No Response   

    What Racial / Ethnic Groups do You Identi

In [910]:
grad = prim.return_color_grad(second, 100)

back = c.color(20, 20, 20)
text = c.color(255, 255, 255)

# Blue
prim = c.color(50, 64, 191)
second = c.color(184, 225, 255)

blue = c.pallet(grad, prim, second, back, back, text)

# Red
prim = c.color(255, 48, 86)
second = c.color(240, 187, 146)

red = c.pallet(grad, prim, second, back, back, text)

# Green
prim = c.color(47, 163, 82)
second = c.color(240, 187, 146)

green = c.pallet(grad, prim, second, back, back, text)


In [911]:
def font(size):
    return dict(
        color="aliceblue",
        family="Monoid",
        size=size
    )

def layout( title ):
 return dict (
    title =  dict(
        font=font(15),
        text=title,
        x = 0.5,
    ),
    legend= dict(
        bgcolor="rgb(30, 30, 40)",
        bordercolor="rgb(20, 20, 30)",
        borderwidth=7,
        font= font(10),
        orientation="v",
        title = dict(
            font=font(15),
            # text="Legend Title",
        ),
    ),
    width = 800,
    height = 500,
    font=font(10),

    paper_bgcolor="rgb(30, 30, 40)",
    plot_bgcolor="rgb(20, 20, 30)",

    yaxis=dict(
        gridcolor="rgb(10, 10,15)",
        gridwidth=3,
        tickangle=90
    ),
    xaxis=dict(
        showgrid=False,
        tickangle=30
    ),
    

    grid=dict(
        columns=1,
        rows=1,
    ),

    bargap=0.2
)

In [912]:
def return_slice_of_elements( element, arr ):
    array = arr.copy()
    start = array.index(element)
    array.reverse()
    end = len(array) - 1 - array.index(element)
    array.reverse()
    return ( start, end )

def get_intersection_name( intersection ):
    rows = intersections["columnName"] == intersection
    return str(intersections.loc[ rows, "intersection" ].item())

In [913]:

def render_graph(question, figure, pallett):
    traces = []
    cooresponding_intersections = []

    options = len(useable_data[question])
    values = np.full( options, 1 )

    buttons = []

    # This will put all the traces into a list
    for i, intersection in enumerate(intersections["columnName"]):
        if not intersection == "blank":
            fig_data = figure(question, intersection, values, i, pallett )
            traces += (fig_data)
            cooresponding_intersections = list(cooresponding_intersections) + list(np.full(len( fig_data ), str(intersection) ).astype(str))

    # This add the buttons and intersections to all these traces 
    

    for i, intersection in enumerate(intersections["columnName"]):
        if not intersection == "blank":
            
            name = get_intersection_name(intersection)
            visibility = np.full( len(traces), False )

            slice = return_slice_of_elements( intersection, cooresponding_intersections) 
            visibility[slice[0]:slice[1] + 1] = True

            buttons.append(dict(label=str(name),
                        method="update",
                        args=[{"visible":visibility}] ))

    fig = go.Figure(data=traces)

    fig.update_layout(
        updatemenus=[
            dict(
                direction="down",
                active=0,
                x=0,
                y=1,
                buttons=list(buttons)
            )
        ])    

    name = question

    if "(" in name:
        start = name.index( "(" )
        end = name.index( ")" )
        name = name[ 0: start ] + name[ end + 1: -1 ]

    fig.update_layout(layout( name ))
    return fig

In [914]:
def color_map( intersections, pallett ):
    returning = {}

    arr = useable_data[intersections].astype(str)

    color1 = pallett.prim
    color2 = pallett.second

    length = len( np.unique(arr) )
    
    i = 1
    for intersection in arr:
        if not intersection in returning:
            color = color1.return_color_between(color2, ( i / length )).return_string_in("RGB")
            key = str( intersection )
            
            # if "." in key: key = key[0: key.index(".")]
            # print(key)
            returning[ key ] = color
            i += 1
            
    return returning

In [915]:
def render_sundial(question, intersection, values, i, pallett):
    return list(px.sunburst(useable_data, 
        path=[question, intersection], 
        values=values,
        color=intersection,
        color_discrete_map = color_map(intersection, pallett),
    
        
    ).update_traces(visible=True if i==0 else False, hovertemplate=None
    ).update_layout(hovermode="closest").data)

def render_bar(question, intersection, values, i, pallett):
    return list(px.bar(useable_data, 
        x=question, 
        y=values, 
        color=intersection,
        color_discrete_map = color_map(intersection, pallett)
    ).update_traces(visible=True if i==0 else False, hovertemplate=None, marker_line_width = 0, selector=dict(type="bar")
    ).update_layout(hovermode="closest", bargap=0, bargroupgap = 0,).data)

In [916]:



prefs = 9
accessability = 16
health = 25

# for i in range(prefs, len(useable_data.columns)):
for i in range(prefs, len(useable_data.columns)):

    pallett = red
    if i > accessability:
        pallett = blue
    if i > health:
        pallett = green

    bar = render_graph(
        useable_data.columns[i], 
        render_bar, pallett)

    sundial = render_graph(
        useable_data.columns[i], 
        render_sundial, pallett)
    
    header = '<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="width=device-width, initial-scale=1.0"><title>Document</title></head><body bgcolor="#1e1e28">'
    div = sundial.to_html(full_html=False, include_plotlyjs='cdn')
    div2 = bar.to_html(full_html=False, include_plotlyjs='cdn')



    with open('/Users/brianmasse/Desktop/p_graph' + str(i) + '.html', 'a') as f:
        f.write(header)
        f.write(div)
        f.write(div2)
        f.write( "</body>" )


What is Your Favorite Meal?
